In [7]:
import csv
import os
import pandas as pd
import math

def haversine(lat1, lon1, lat2, lon2):
   # Convert latitude and longitude from degrees to radians
   lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

   dlat = lat2 - lat1
   dlon = lon2 - lon1
   a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
   c = 2 * math.asin(math.sqrt(a))

   # Radius of Earth in kilometers. Use 3956 for miles.
   r = 6371

   # Calculate the result
   distance = c * r
   return distance

# Get the current working directory 
current_directory = os.getcwd() 

# Print the current working directory 
print(current_directory)

filename = current_directory + '\\locations_output.csv'

df1 = pd.read_csv(filename)

filename = current_directory + '\\WMO_Weather_Stations_Combined.csv'

df2 = pd.read_csv(filename)

print(df1)
print(df2)

closest_distance = []

for i in range(106):
   distances = []
   file_id = df1['file_no'].values[i]
   print("Location: "+str(i)+", "+str(file_id))
   lat_h = df1['latitude'].values[i]
   lon_h = df1['longitude'].values[i]
   lat_min = lat_h - 0.5
   lat_max = lat_h + 0.5
   lon_min = lon_h - 0.5
   lon_max = lon_h + 0.5
   for j in range(df2.shape[0]):
      lat_noaa = df2['Latitude'].values[j]
      lon_noaa = df2['Longitude'].values[j]
      location = df2['WMO_ID'].values[j]
      if(lat_noaa < lat_max and lat_noaa > lat_min and lon_noaa > lon_min and lon_noaa < lon_max):
         distances.append([file_id, lat_h, lon_h, location, lat_noaa, lon_noaa, haversine(lat_h, lon_h, lat_noaa, lon_noaa)])

   if(distances == []):
      closest_distance.append([file_id, lat_h, lon_h, NaN, NaN, NaN, NaN])
   else:
      distances2 = pd.DataFrame(distances)
      distances2.columns = ['file_id', 'lat_h', 'lon_h', 'location', 'lat_noaa', 'lon_noaa', 'distance']
      cdistance = distances2['distance'].values[0]
      cind = 0 # index of closest distance
      for j in range(distances2.shape[0]):
         if(cdistance > distances2['distance'].values[j]):
            cdistance = distances2['distance'].values[j] # new closest distance
            cind = j
      closest_distance.append([distances2['file_id'].values[cind], distances2['lat_h'].values[cind], distances2['lon_h'].values[cind], 
      distances2['location'].values[cind], distances2['lat_noaa'].values[cind], distances2['lon_noaa'].values[cind], distances2['distance'].values[cind]])

print(closest_distance)

C:\Users\User
     file_no  latitude  longitude  elevation    nobs     min     max   range  \
0       6501    33.636    -91.756       85.0  174306  648697  823269  174572   
1       6541    38.958    -94.371      304.8  179295  643403  823269  179866   
2       6640    28.474    -82.454       19.7  226660  596088  823269  227181   
3       6668    35.140    -90.236       63.7  181069  641806  823269  181463   
4       6678    30.393    -86.467        5.4  244219  578568  823269  244701   
..       ...       ...        ...        ...     ...     ...     ...     ...   
103     8184    36.985   -120.111       76.6  226726  596088  823269  227181   
104     8186    33.038   -116.916      422.0  226859  596088  823269  227181   
105     8247    30.337    -81.513       11.9  173970  648697  823269  174572   
106     8248    26.079    -80.162        1.0  174022  648696  823269  174573   
107     9858    36.118    -97.091      271.3  199033  624085  823271  199186   

     missing values  regi

In [9]:
df = pd.DataFrame(closest_distance)

# Write DataFrame to CSV
df.to_csv('HADISDH_NOAA_Weather_Station_Matches.csv', index=False)